In [1]:
from contextlib import contextmanager
import os
import datetime as dt
import numpy as np
import h5py
import time
import sys
os.chdir('d://software_git_repos/greenpol/telescope_control/data_aquisition/')
sys.path.append('../')
sys.path.append('d:/software_git_repos/cofe-python-analysis-tools/utils_meinhold/')
sys.path.append('d:/software_git_repos/cofe-python-analysis-tools/utils_zonca/')
from glob import glob

sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('../VtoT/')

import Tkinter,tkFileDialog

import cofe_util as cu
import realtime_gp as rt
import datetime

from Tkinter import *
import ttk

In [2]:
def nametochan(name):
    #function to convert channel numbers to channel names

    #names of each channel
    chans = {
        'all': 'all',  'H1HiAC':'ch0',  'H1HiDC':'ch1',
        'H1LoAC':'ch2' ,  'H1LoDC':'ch3', 'H2HiAC':'ch4' ,
        'H2HiDC':'ch5' ,  'H2LoAC':'ch6',  'H2LoDC':'ch7',
        'H3HiAC':'ch8', 'H3HiDC':'ch9',  'H3LoAC':'ch10',
         'H3LoDC':'ch11', 'HornTop':'ch12', 'Amplifier': 'ch13',
        'Cooler':'ch14', 'Transition':'ch15'}

    chan = chans[name]

    return chan

In [3]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\labuser\Anaconda\envs\py27\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Widget', 'Text', 'Button']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
def plot_some_chans2(plottype='toi',filelist=None,component='T',samprate=30,minfreq=.1):
    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='d://software_git_repos/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    main = Tk()
    main.title("Choose channels to plot")
    main.geometry("+50+150")
    frame = ttk.Frame(main, padding=(3, 3, 12, 12))
    frame.grid(column=0, row=0, sticky=(N, S, E, W))

    #set up the listbox to choose what chans to plot
    chan_labels = StringVar()
    chan_labels.set("H1HiAC H1HiDC H1LoAC H1LoDC H2HiAC H2HiDC H2LoAC H2LoDC\
                 H3HiAC H3HiDC H3LoAC H3LoDC HornTop Amplifier Cooler Transition")           
    dlist=[]
    for f in filelist[1:]:
        hf=h5py.File(f)
        dlist.append(hf['demod_data'])
    d=concatenate(dlist)
    hf.close() 
    
    lstbox = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox.grid(column=0, row=0, columnspan=2)
    
    lstbox2 = Listbox(frame,listvariable=chan_labels, selectmode=MULTIPLE, width=20, height=16)
    lstbox2.grid(column=2, row=0, columnspan=2)
    #start the plot

    fname=filelist[0]
    yyyy=os.path.dirname(filelist[0])[-8:-4]
    mm=os.path.dirname(filelist[0])[-4:-2]
    dd=os.path.dirname(filelist[0])[-2:]

    def select():
        reslist = list()
        selection = lstbox.curselection()
        figure()
        title('%s TOI from %s %s %s' %(component,mm,dd,yyyy))

        for i in selection:
            entry = lstbox.get(i)
            reslist.append(entry)
        for val in reslist:
            chan=nametochan(val)
            plot(d['rev'],d[chan][component],label=val)
            xlabel('GPS time [mS]')
            ylabel('output [v]')
            legend()

    def selectfft():
        reslist2 = list()
        selection2 = lstbox2.curselection()
        figure()
        title('%s ASD from %s %s %s' %(component,mm,dd,yyyy))

        for i in selection2:
            entry2 = lstbox2.get(i)
            reslist2.append(entry2)
        for val in reslist2:
            chan=nametochan(val)
            freq,psd=cu.nps(d[chan][component],samprate,minfreq=minfreq)
            plot(freq,sqrt(psd)*1e9,label=val)
            xlabel('Frequency [Hz]')
            ylabel(r'ASD [$\frac{nV}{\sqrt{Hz}}$]')
            legend()
                
    btn = ttk.Button(frame, text="SelectTOI", command=select)
    btn.grid(column=1, row=1)
    
    btn2 = ttk.Button(frame, text="SelectFFT", command=selectfft)
    btn2.grid(column=3, row=1)
    
    main.mainloop()

In [ ]:
plot_some_chans2(component='T')